## 生成器

通过列表生成式，我们可以直接创建一个列表。但是，受到内存限制，列表容量肯定是有限的。而且，创建一个包含100万个元素的列表，不仅占用很大的存储空间，如果我们仅仅需要访问前面几个元素，那后面绝大多数元素占用的空间都白白浪费了。

所以，如果列表元素可以按照某种算法推算出来，那我们是否可以在循环的过程中不断推算出后续的元素呢？这样就不必创建完整的list，从而节省大量的空间。在Python中，这种一边循环一边计算的机制，称为生成器：generator。

### 创建 generator 的两种方式
#### 第一种：把一个列表生成式的[]改成()

In [9]:
L = [x * x for x in range(5)]
g = (x * x for x in range(5))
print(L)
print(g)

[0, 1, 4, 9, 16]
<generator object <genexpr> at 0x00A09ED0>


创建L和g的区别仅在于最外层的[]和()，L是一个list，而g是一个generator。

我们可以直接打印出list的每一个元素，但我们怎么打印出generator的每一个元素呢？

如果要一个一个打印出来，可以通过next()函数获得generator的下一个返回值：

##### 通过next()函数获得generator的下一个返回值

In [10]:
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

0
1
4
9
16


StopIteration: 

**generator保存的是算法**，每次调用next(g)，就计算出g的下一个元素的值，直到计算到最后一个元素，没有更多的元素时，抛出StopIteration的错误。

当然，上面这种不断调用next(g)实在是太变态了，正确的方法是使用for循环，因为generator也是可迭代对象：

In [11]:
g = (x * x for x in range(5))
for i in g:
    print(i)

0
1
4
9
16


所以，我们创建了一个generator后，基本上永远不会调用next()，而是通过for循环来迭代它，并且不需要关心StopIteration的错误。


#### 第二种：用 yield 
如果一个函数定义中包含yield关键字，那么这个函数就不再是一个普通函数，而是一个generator

以斐波拉契数为例，可以输出斐波那契数列的前N个数

In [12]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        print(b)
        a, b = b, a + b
        n = n + 1
    return 'done'

fib(4)

1
1
2
3


'done'

仔细观察，可以看出，fib函数实际上是定义了斐波拉契数列的推算规则，可以从第一个元素开始，推算出后续任意的元素，这种逻辑其实非常类似generator。

也就是说，上面的函数和generator仅一步之遥。要把fib函数变成generator，只需要把print(b)改为yield b就可以了：

In [18]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

f = fib(4)
print(f)
for i in f:
    print(i)

<generator object fib at 0x00A049C0>
1
1
2
3


这里，最难理解的就是generator和函数的执行流程不一样。函数是顺序执行，遇到return语句或者最后一行函数语句就返回。
**而变成generator的函数，在每次调用 next()的时候执行，遇到yield语句返回，再次执行时从上次返回的yield语句处继续执行。**

In [19]:
def odd():
    print('step 1')
    yield 1
    print('step 2')
    yield(3)
    print('step 3')
    yield(5)
    
o = odd()
print(next(o))
print(next(o))
print(next(o))
print(next(o))

step 1
1
step 2
3
step 3
5


StopIteration: 

odd不是普通函数，而是generator，在执行过程中，遇到yield就中断，下次又继续执行。执行3次yield后，已经没有yield可以执行了，所以，第4次调用next(o)就报错。

In [22]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

g = fib(4)
for n in fib(4):
    print(n)

1
1
2
3


**但是用for循环调用generator时，发现拿不到generator的return语句的返回值。**

如果想要拿到返回值，必须捕获StopIteration错误，返回值包含在StopIteration的value中：

In [29]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

g = fib(4)

print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

1
1
2
3


StopIteration: done

In [31]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

g = fib(4)
while True:
    try:
        x = next(g)
        print(x)
    except StopIteration as e:
        print('Generator return value:',e.value)
        break

1
1
2
3
Generator return value: done
